In [13]:
import numpy as np
import numpy.random as rnd
import scipy.special
import scipy.optimize
from scipy.optimize import fsolve
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib
import matplotlib.font_manager as fm
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
cmap = plt.get_cmap('viridis')

# This is a notebook for similating the ODEs and performing numerical experiments
### The first part defines helper functions the second goes through different reward settings and defines functions for numerically integrating the ODEs and correspondingly performing the synthetic simulations

In [3]:
"""
Returns the test accuracy for a teacher-student pair with the given overlaps.
"""
def p_T_correct(Q, R, T):
    return (1 - 1 / np.pi * np.arccos(R / np.sqrt(Q)))**T

"""
generates random vector in D dimensions with magnitude sqrt{D}
"""
def gen_perceptron(D):
    teacher = rnd.randn(D)
    teacher /= np.sqrt(teacher @ teacher/D)
    return teacher

Set hyperparameters


In [ ]:
D = 400
lr_1 = 1
T = 10
steps = 800
teacher = gen_perceptron(D)
student = gen_perceptron(D)

## All decisions correct required for reward

In [ ]:
def all_neg(D, teacher, student, T, lr_1, lr_2, steps):

    R = teacher @ student / D
    Q = student @ student / D

    data = dict()
    data['r'] = []
    data['q'] = []

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % D == 0:
            data['r'].append(np.around(copy.deepcopy(R), 5))
            data['q'].append(np.around(copy.deepcopy(Q), 5))
        normalised_overlap = np.divide(np.copy(R), np.sqrt(np.copy(Q)))
        theta = np.arccos(normalised_overlap)
        p_correct_all = (1 - 1 / np.pi * np.arccos(normalised_overlap)) ** T

        phi = (np.pi - theta) / 2
        C_1 = np.sqrt(np.pi / 2) * np.sin(phi) / phi

        half_overlap = np.sqrt(1 + normalised_overlap)

        # compute r,q updates
        dR = (lr_1 + lr_2) * C_1 / np.sqrt(2) * p_correct_all * half_overlap - lr_2 * R * np.sqrt(2 / (Q * np.pi))

        dQ = lr_2 ** 2 / (T * D) * (D + (T - 1) * 2 / np.pi) - 2 * lr_2 * np.sqrt(2 * Q / np.pi) + (
                    (lr_1 ** 2 - lr_2 ** 2) * (D + (T - 1) * C_1 ** 2) / (T * D) + (lr_1 + lr_2) * np.sqrt(
                2 * Q) * half_overlap * C_1) * p_correct_all

        # update r, q
        R += dt * dR
        Q += dt * dQ

        step += 1

    data['r'].append(np.around(copy.deepcopy(R), 5))
    data['q'].append(np.around(copy.deepcopy(Q), 5))

    data['r'] = np.asarray(data['r'])
    data['q'] = np.asarray(data['q'])

    return data

def all_neg_exp(D, teacher, student, T, lr_1, lr_2, steps):
    cp.cuda.Device(0).use()
    teacher = cp.asarray(teacher)
    student = cp.asarray(student)

    teachers = cp.tile(cp.expand_dims(teacher, axis=0), (20, 1)) 

    W = cp.tile(cp.expand_dims(student, axis=0), (20, 1))

    data = dict()
    data['R'] = cp.zeros((1, 20))
    data['Q'] = cp.zeros((1, 20))
    

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % (D) == 0:
            R = cp.sum(teachers * cp.copy(W), axis=1) / D
            Q = cp.sum(cp.copy(W) ** 2, axis=1) / D
            
            data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
            data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

        X = rnd.randn(T, 20, D)
        # predicted classification
        Y_pred = cp.sign(cp.sum(cp.expand_dims(cp.copy(W), axis=0) * X, axis=2))
        
        # actual classification
        Y = cp.sign(cp.sum(cp.expand_dims(cp.copy(teachers), axis=0) * X, axis=2))
        
        reward = cp.all(Y_pred == Y, axis=0)
        reward = cp.expand_dims(reward, axis=1)

        hebbian_update = cp.mean(cp.expand_dims(Y_pred, axis=2) * X, axis=0)

        W += ((lr_1 + lr_2) * reward - lr_2) * hebbian_update / cp.sqrt(D)

        step += 1

    R = cp.sum(teachers * cp.copy(W), axis=1) / D
    Q = cp.sum(cp.copy(W) ** 2, axis=1) / D

    data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
    data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

    data['R'] = cp.asnumpy(data['R'])
    data['Q'] = cp.asnumpy(data['Q'])

    return data

#### Plot

In [ ]:

"""
set lr_2 here
"""
lr_2 = 0
ode_dat = all_neg(D, teacher, student, T, lr_1, lr_2, steps)
sim_dat = all_neg_exp(D, teacher, student, T, lr_1, lr_2, steps)

fig, ((R_ax, Q_ax)) = plt.subplots(2, figsize = (10,5) , constrained_layout = True)

R_ax.fill_between(np.arange(ode_dat['r'].shape), np.mean(sim_dat['R'], axis = 1) - np.std(sim_dat['R'], axis = 1), np.mean(sim_dat['R'], axis = 1) + np.std(sim_dat['R'], axis = 1))
R_ax.plot(ode_dat['r'], '--', color = 'black', alpha = 0.7)

Q_ax.fill_between(np.arange(ode_dat['q'].shape), np.mean(sim_dat['Q'], axis = 1) - np.std(sim_dat['Q'], axis = 1), np.mean(sim_dat['Q'], axis = 1) + np.std(sim_dat['Q'], axis = 1))
Q_ax.plot(ode_dat['q'], '--', color = 'black', alpha = 0.7)

R_ax.set_xlabel('time')
Q_ax.set_xlabel('time')
R_ax.set_ylabel(r'$R$')
Q_ax.set_ylabel(r'$Q$')

## n or more decisions correct required for reward

In [6]:
def n_or_more_neg(D, teacher, student, T, n, lr_1, lr_2, steps):

    R = teacher @ student / D
    Q = student @ student / D

    data = dict()
    data['r'] = []
    data['q'] = []

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % (D) == 0:
            data['r'].append(np.around(copy.deepcopy(R), 5))
            data['q'].append(np.around(copy.deepcopy(Q), 5))

        normalised_overlap = np.divide(np.copy(R), np.sqrt(np.copy(Q)))
        theta = np.arccos(normalised_overlap)
        p_correct = (1 - theta / np.pi)
        phi = (np.pi - theta) / 2

        C_2 = np.sqrt(np.pi / 2) * np.divide(np.sin(theta / 2), (theta / 2))
        C_1 = np.sqrt(np.pi / 2) * np.divide(np.sin(phi), phi)

        half_overlap = np.sqrt(1 + normalised_overlap)
        half_incorrect = np.sqrt(1 - normalised_overlap)

        a = 0
        b = 0
        c = 0
        d = 0
        e = 0

        for i in range(n, T + 1):
            p_i = p_correct ** i
            q_i = (1 - p_correct) ** (T - i)
            a += scipy.special.binom(T, i) * i * p_i * q_i
            b += scipy.special.binom(T, i) * (T - i) * p_i * q_i

            c += scipy.special.binom(T, i) * p_i * q_i
            d += scipy.special.binom(T, i) * i * (i - 1) * p_i * q_i
            e += scipy.special.binom(T, i) * (T - i) * (T - i - 1) * p_i * q_i

        # compute r,q updates
        dR = (lr_1 + lr_2) / (T * np.sqrt(D)) * (a * C_1 * np.sqrt(D / 2) * half_overlap - b * C_2 * np.sqrt(
            D / 2) * half_incorrect) - lr_2 * np.sqrt(2 / np.pi) * normalised_overlap

        dQ = (2 * (lr_1 + lr_2) / (T * np.sqrt(D)) * (
                    a * C_1 * np.sqrt(D * Q / 2) * half_overlap + b * C_2 * np.sqrt(D * Q / 2) * half_incorrect) +
              (lr_1 ** 2 - lr_2 ** 2) / (T ** 2 * D) * (c * T * D + d * C_1 ** 2 + e * C_2 ** 2)) - 2 * lr_2 * np.sqrt(
            2 * Q / np.pi) + lr_2 ** 2 / (T * D) * (D + (T - 1) * 2 / np.pi)

        # update r, q
        R += dt * dR
        Q += dt * dQ
        step += 1

    data['r'].append(np.around(copy.deepcopy(R), 5))
    data['q'].append(np.around(copy.deepcopy(Q), 5))
    
    data['r'] = np.asarray(data['r'])
    data['q'] = np.asarray(data['q'])

    return data

def n_or_more_neg_exp(D, teacher, rad, student, T, n, lr_1, lr_2, steps, experiment_path):
    cp.cuda.Device(0).use()
    teacher = cp.asarray(teacher)
    student = cp.asarray(student)

    teachers = cp.tile(cp.expand_dims(teacher, axis=0), (20, 1))
    
    path = os.path.join(experiment_path, f'exp_{T}-{n}-{rad}-{lr_2}')
    os.mkdir(path)

    # initialize all students
    W = cp.tile(cp.expand_dims(student, axis=0), (20, 1)) 

    # create dictionary of order parameters
    data = dict()
    data['R'] = cp.zeros((1, 20))
    data['Q'] = cp.zeros((1, 20))

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % (D) == 0:
            print(step)
            R = cp.sum(teachers * cp.copy(W), axis=1) / D
            Q = cp.sum(cp.copy(W) ** 2, axis=1) / D

            data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
            data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

        # sample T examples
        X = rnd.randn(T, 20, D)

        # predicted classification
        Y_pred = cp.sign(cp.sum(cp.expand_dims(cp.copy(W), axis=0) * X, axis=2))
        
        # actual classification
        Y = cp.sign(cp.sum(cp.expand_dims(cp.copy(teachers), axis=0) * X, axis=2))
        
        # create filter for rewards (1/0)
        reward = Y * Y_pred + 1
        reward = cp.sum(reward, axis=0)
        reward = reward >= 2 * n
        reward = reward.astype(int)
        reward = cp.expand_dims(reward, axis=1)

        # update from mean of examples over episode
        hebbian_update = cp.mean(cp.expand_dims(Y_pred, axis=2) * X, axis=0)
        
        # update students
        W += ((lr_1 + lr_2) * reward - lr_2) * hebbian_update / cp.sqrt(D)

        step += 1

    R = cp.sum(teachers * cp.copy(W), axis=1) / D
    Q = cp.sum(cp.copy(W) ** 2, axis=1) / D

    data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
    data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

    data['R'] = cp.asnumpy(data['R'])
    data['Q'] = cp.asnumpy(data['Q'])

    return data

#### Plot

In [ ]:
n = 8
ode_dat = n_or_more_neg(D, teacher, student, T, n, lr_1, lr_2, steps)
sim_dat = n_or_more_neg_exp(D, teacher, student, T, n, lr_1, lr_2, steps)

fig, ((R_ax, Q_ax)) = plt.subplots(2, figsize = (10,5) , constrained_layout = True)

R_ax.fill_between(np.arange(ode_dat['r'].shape), np.mean(sim_dat['R'], axis = 1) - np.std(sim_dat['R'], axis = 1), np.mean(sim_dat['R'], axis = 1) + np.std(sim_dat['R'], axis = 1))
R_ax.plot(ode_dat['r'], '--', color = 'black', alpha = 0.7)

Q_ax.fill_between(np.arange(ode_dat['q'].shape), np.mean(sim_dat['Q'], axis = 1) - np.std(sim_dat['Q'], axis = 1), np.mean(sim_dat['Q'], axis = 1) + np.std(sim_dat['Q'], axis = 1))
Q_ax.plot(ode_dat['q'], '--', color = 'black', alpha = 0.7)

R_ax.set_xlabel('time')
Q_ax.set_xlabel('time')
R_ax.set_ylabel(r'$R$')
Q_ax.set_ylabel(r'$Q$')

## Survival up to n timesteps required for additional reward of size of size lr_2

In [8]:
def partial_ode(D, teacher, student, T, n, lr_1, lr_2, steps):

    R = teacher @ student / D
    Q = student @ student / D

    data = dict()
    data['r'] = []
    data['q'] = []

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % (D) == 0:
            data['r'].append(np.around(copy.deepcopy(R), 5))
            data['q'].append(np.around(copy.deepcopy(Q), 5))

        normalised_overlap = np.divide(np.copy(R), np.sqrt(np.copy(Q)))
        p_correct = (1 - 1 / np.pi * np.arccos(normalised_overlap))

        # compute r,q updates
        dR = 1 /(T * np.sqrt(2 * np.pi)) * (1 + normalised_overlap) * p_correct ** (n - 1) * (T * lr_1 * p_correct ** (T - n) + lr_2 * n)

        dQ = (p_correct ** (n - 1) / T * np.sqrt(2 * Q / np.pi) * (1 + normalised_overlap) * (
                    lr_1 * T * p_correct ** (T - n) + lr_2 * n)
              + p_correct ** n / T ** 2 * ((lr_1 * T + 2 * n * lr_2 * lr_1) * p_correct ** (T - n) + lr_2 ** 2 * n))
       
        # update r, q
        R += dt * dR
        Q += dt * dQ
    
        step += 1

    data['r'].append(np.around(copy.deepcopy(R), 5))
    data['q'].append(np.around(copy.deepcopy(Q), 5))
    
    data['r'] = np.asarray(data['r'])
    data['q'] = np.asarray(data['q'])

    return data


def partial_exp(D, teacher, student, T, n, lr_1, lr_2, steps):
    cp.cuda.Device(0).use()
    teacher = cp.asarray(teacher)
    student = cp.asarray(student)

    teachers = cp.tile(cp.expand_dims(teacher, axis=0), (20, 1))

    # initialize all students
    W = cp.tile(cp.expand_dims(student, axis=0), (20, 1))
    
    # create dictionary of order parameters
    data = dict()
    data['R'] = cp.zeros((1, 20))
    data['Q'] = cp.zeros((1, 20))
    

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % (D) == 0:
            R = cp.sum(teachers * cp.copy(W), axis=1) / D
            Q = cp.sum(cp.copy(W) ** 2, axis=1) / D
            data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
            data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

        # sample T examples
        X = rnd.randn(T, 20, D)
        # predicted classification
        Y_pred = cp.sign(cp.sum(cp.expand_dims(cp.copy(W), axis=0) * X, axis=2))
        Y = cp.sign(cp.sum(cp.expand_dims(cp.copy(teachers), axis=0) * X, axis=2)) 
        
        reward_1 = lr_1 * cp.all(Y_pred == Y, axis=0)
        reward_2 = lr_2 * cp.all(Y_pred[:n, :] == Y[:n, :], axis=0)
        reward = cp.zeros_like(Y_pred)
        reward[:n, :] += reward_2
        reward += reward_1

        hebbian_update = cp.mean(cp.expand_dims(Y_pred, axis=2) * X * cp.expand_dims(reward, axis=2), axis=0)

        W += hebbian_update / cp.sqrt(D)

        # log order parameters
        step += 1

    R = cp.sum(teachers * cp.copy(W), axis=1) / D
    Q = cp.sum(cp.copy(W) ** 2, axis=1) / D

    data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
    data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

    data['R'] = cp.asnumpy(data['R'])
    data['Q'] = cp.asnumpy(data['Q'])

    return data

#### Plot

In [9]:
T_0 = 6
ode_dat = partial_ode(D, teacher, student, T, T_0, lr_1, lr_2, steps)
sim_dat = partial_exp(D, teacher, student, T, T_0, lr_1, lr_2, steps)

fig, ((R_ax, Q_ax)) = plt.subplots(2, figsize = (10,5) , constrained_layout = True)

R_ax.fill_between(np.arange(ode_dat['r'].shape), np.mean(sim_dat['R'], axis = 1) - np.std(sim_dat['R'], axis = 1), np.mean(sim_dat['R'], axis = 1) + np.std(sim_dat['R'], axis = 1))
R_ax.plot(ode_dat['r'], '--', color = 'black', alpha = 0.7)

Q_ax.fill_between(np.arange(ode_dat['q'].shape), np.mean(sim_dat['Q'], axis = 1) - np.std(sim_dat['Q'], axis = 1), np.mean(sim_dat['Q'], axis = 1) + np.std(sim_dat['Q'], axis = 1))
Q_ax.plot(ode_dat['q'], '--', color = 'black', alpha = 0.7)

R_ax.set_xlabel('time')
Q_ax.set_xlabel('time')
R_ax.set_ylabel(r'$R$')
Q_ax.set_ylabel(r'$Q$')

NameError: name 'teacher' is not defined

## Every correct decision gives an additional reward of size lr_2

In [10]:
def bread_discount_ode(D, teacher, student, T, lr_1, lr_2, steps):
    
    R = teacher @ student / D
    Q = student @ student / D

    data = dict()
    data['r'] = []
    data['q'] = []

    step = 0
    num_steps = steps * D
    dt = 1 / D

    while step < num_steps:
        if step % (D) == 0:
            data['r'].append(np.around(copy.deepcopy(R), 5))
            data['q'].append(np.around(copy.deepcopy(Q), 5))

        normalised_overlap = np.divide(np.copy(R), np.sqrt(np.copy(Q)))
        p_correct = (1 - 1 / np.pi * np.arccos(normalised_overlap))

        # compute r,q updates
        dR = 1 / np.sqrt(2 * np.pi) * ((1 + normalised_overlap) * (lr_1 * p_correct ** (T - 1) + lr_2) + lr_2 * (
                    T - 1) * normalised_overlap * p_correct)

        dQ = np.sqrt(2 * Q / np.pi) * ((1 + normalised_overlap) * (lr_1 * p_correct ** (T - 1) + lr_2) + lr_2 * (
                    T - 1) * p_correct) + lr_1/T * (
                         lr_1 + lr_2*(T+1)) * p_correct ** T + lr_2 ** 2 * (T+1)/T * (1/2 + (T - 1) * p_correct/3) * p_correct

        # update r, q
        R += dt * dR
        Q += dt * dQ

        step += 1

    data['r'].append(np.around(copy.deepcopy(R), 5))
    data['q'].append(np.around(copy.deepcopy(Q), 5))
    
    data['r'] = np.asarray(data['r'])
    data['q'] = np.asarray(data['q'])

    return data

def bread_discount_exp(D, teacher, student, T, lr_1, lr_2, steps):
    cp.cuda.Device(0).use()
    teacher = cp.asarray(teacher)
    student = cp.asarray(student)

    teachers = cp.tile(cp.expand_dims(teacher, axis=0), (20, 1))
    
    # initialize all students
    W = cp.tile(cp.expand_dims(student, axis=0), (20, 1))

    # create dictionary of order parameters
    data = dict()
    data['R'] = cp.zeros((1, 20))
    data['Q'] = cp.zeros((1, 20))

    step = 0
    num_steps = steps * D
    dt = 1 / D

    def create_upper_matrix(size, value=1):
        upper = cp.zeros((size, size))
        upper[cp.triu_indices(size, 0)] = value
        return (upper)

    discount_matrix = create_upper_matrix(T)

    while step < num_steps:
        if step % (D) == 0:
            R = cp.sum(teachers * cp.copy(W), axis=1) / D
            Q = cp.sum(cp.copy(W) ** 2, axis=1) / D
            data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
            data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

        # sample T examples
        X = rnd.randn(T, 20, D)

        # predicted classification
        Y_pred = cp.sign(cp.sum(cp.expand_dims(cp.copy(W), axis=0) * X, axis=2))
        # actual classification
        Y = cp.sign(cp.sum(cp.expand_dims(cp.copy(teachers), axis=0) * X, axis=2))
        
        r = (Y_pred*Y + 1)/2
        g = lr_2 * discount_matrix @ r
        discounted_reward = g + lr_1*cp.expand_dims(np.all(Y_pred == Y, axis=0), axis=0)
        
        # update from mean of examples over episode
        hebbian_update = cp.mean(cp.expand_dims(Y_pred, axis = 2) * X * cp.expand_dims(discounted_reward, axis = 2), axis = 0)
        
        # update students
        W += hebbian_update / cp.sqrt(D)

        # log order parameters

        step += 1

    R = cp.sum(teachers * cp.copy(W), axis=1) / D
    Q = cp.sum(cp.copy(W) ** 2, axis=1) / D

    data['R'] = cp.concatenate((data['R'], cp.expand_dims(cp.around(copy.deepcopy(R), 5), 0)), axis=0)
    data['Q'] = cp.concatenate((data['Q'], cp.expand_dims(cp.around(copy.deepcopy(Q), 5), 0)), axis=0)

    data['R'] = cp.asnumpy(data['R'])
    data['Q'] = cp.asnumpy(data['Q'])
    
    return data

#### Plot

In [ ]:
r_b = 0.01

ode_dat = bread_discount_ode(D, teacher, student, T, T_0, lr_1, r_b, steps)
sim_dat = bread_discount_exp(D, teacher, student, T, T_0, lr_1, r_b, steps)

fig, ((R_ax, Q_ax)) = plt.subplots(2, figsize = (10,5) , constrained_layout = True)

R_ax.fill_between(np.arange(ode_dat['r'].shape), np.mean(sim_dat['R'], axis = 1) - np.std(sim_dat['R'], axis = 1), np.mean(sim_dat['R'], axis = 1) + np.std(sim_dat['R'], axis = 1))
R_ax.plot(ode_dat['r'], '--', color = 'black', alpha = 0.7)

Q_ax.fill_between(np.arange(ode_dat['q'].shape), np.mean(sim_dat['Q'], axis = 1) - np.std(sim_dat['Q'], axis = 1), np.mean(sim_dat['Q'], axis = 1) + np.std(sim_dat['Q'], axis = 1))
Q_ax.plot(ode_dat['q'], '--', color = 'black', alpha = 0.7)

R_ax.set_xlabel('time')
Q_ax.set_xlabel('time')
R_ax.set_ylabel(r'$R$')
Q_ax.set_ylabel(r'$Q$')